In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import os

# Create a datfaframe to hold facets sensor location data
sensor_location_data = {'Sensor ID': [], 'Sensor Description': [], 'Status': [], 'Location Type': [], 'Longitude': [], 'Latitude': []}
sensor_location_df = pd.DataFrame(sensor_location_data)

sensor_location_df

In [ ]:
# Set the API base URL
url = 'https://data.melbourne.vic.gov.au/api/'

# Define the endpoint and query parameters
endpoint = '/records/1.0/search/'
params = {
    'dataset': 'pedestrian-counting-system-sensor-locations',
    'q': '',
    'facet': 'location_id'
}

# Send the API request to get the list of distinct Sensor IDs
response = requests.get(url + endpoint, params=params)
data = response.json()
sensor_ids = [f['name'] for f in data['facet_groups'][0]['facets']]

# Create an empty list to store the results
results = []

# Loop through the Sensor IDs to retrieve the data for each sensor
for sensor_id in sensor_ids:
    params['q'] = f'location_id:"{sensor_id}"'
    response = requests.get(url + endpoint, params=params)
    data = response.json()
    records = data['records']

    for record in records:
        sensor_location_data = {
            'Sensor ID': record['fields']['location_id'],
            'Sensor Description': record['fields']['sensor_description'],
            'Status': record['fields']['status'],
            'Location Type': record['fields']['location_type'],
            'Longitude': record['fields']['longitude'],
            'Latitude': record['fields']['latitude']
        }
        results.append(sensor_location_data)

# Convert the results to a DataFrame
sensor_location_df = pd.DataFrame(results)

# Print summary
print("Data Retrieval Complete")
print(f"Number of Records: {len(sensor_location_df)}")

filename = '../Datasets/sensor_locations.csv'
sensor_location_df.to_csv(filename, index=False)

# Display the dataframe
sensor_location_df


In [ ]:
import hvplot.pandas
import geoviews as gv
import geoviews.feature as gf
from geoviews import opts


sensors_map = sensor_location_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "CartoLight",
    frame_width = 800,
    frame_height = 600,
    color = "Status",
    alpha = 0.5,
    hover_cols=["Sensor ID", "Sensor Description"]
)
print(f"Number of sensors in the list: {len(sensor_location_df)}")
sensors_map

In [ ]:
# Pedestrian sensor data file
pedestrian_sensor_counts_path = "../Datasets/Pedestrian_Counting_System_Monthly_counts_per_hour_may_2009_to_14_dec_2022.csv"

# Read the pedestrian sensor data file
pedestrian_sensor_counts = pd.read_csv(pedestrian_sensor_counts_path)


pedestrian_sensor_counts_df = pd.DataFrame(pedestrian_sensor_counts)
pedestrian_sensor_counts_df

In [ ]:
# group the rows by Sensor_ID, Year, Month, and Mdate columns
grouped_df = pedestrian_sensor_counts_df.groupby(['Sensor_ID', 'Year', 'Month', 'Mdate'])

# aggregate the data using the sum function
summed_df = grouped_df.sum(numeric_only=True)

# reset the index to make the columns Sensor_ID, Year, Month, and Mdate regular columns
summed_df = summed_df.reset_index()

# filter the rows based on the given conditions
filtered_df = summed_df[(summed_df['Mdate'] == 1) & (summed_df['Month'] == 'June') & ((summed_df['Year'] == 2018) | (summed_df['Year'] == 2022))]

# reset the index
filtered_df = filtered_df.reset_index(drop=True)

filtered_df = filtered_df.drop(['Time', 'ID'], axis=1)

filtered_df = filtered_df.rename(columns={'Hourly_Counts': 'Daily Count'})
filtered_df = filtered_df.rename(columns={'Sensor_ID': 'Sensor ID'})

filtered_df

In [ ]:
merged_df = pd.merge(filtered_df, sensor_location_df, on='Sensor ID')

# filter the DataFrame for 2018 and 2022
df_2018 = merged_df[merged_df['Year'] == 2018]
df_2022 = merged_df[merged_df['Year'] == 2022]

# group the DataFrames by Sensor_ID
grouped_df_2018 = df_2018.groupby('Sensor ID')
grouped_df_2022 = df_2022.groupby('Sensor ID')

# compute the daily count for each Sensor_ID and year
count_2018 = grouped_df_2018['Daily Count'].sum().reset_index()
count_2022 = grouped_df_2022['Daily Count'].sum().reset_index()

# merge the counts into a single DataFrame
diff_df = pd.merge(count_2022, count_2018, on='Sensor ID', suffixes=('_2022', '_2018'))

# compute the daily count difference for each Sensor_ID
diff_df['Daily Count_Diff'] = diff_df['Daily Count_2022'] - diff_df['Daily Count_2018']

# create new column with percentage change
diff_df['Daily_Count_Diff_Pct'] = 100 * (diff_df['Daily Count_Diff'] / diff_df['Daily Count_2018'])

# round the percentage change to two decimal places
diff_df['Daily_Count_Diff_Pct'] = diff_df['Daily_Count_Diff_Pct'].round(2)

# merge diff_df with sensor_location_df
merged_diff_df = pd.merge(diff_df, sensor_location_df, on='Sensor ID')

# select and reorder columns
merged_diff_df = merged_diff_df[['Sensor ID', 'Sensor Description', 'Latitude', 'Longitude', 
                                 'Daily Count_2022', 'Daily Count_2018', 'Daily Count_Diff', 'Daily_Count_Diff_Pct']]

merged_diff_df


In [ ]:
# Define a scaling function for the size of the points based on the magnitude of the negative values
def scale_size(x):
    if x < 0:
        return np.abs(x) * 10
    else:
        return x * 10

# Define a function to map the color based on the sign of the value
def color_map(x):
    if x < 0:
        return 'red'
    else:
        return 'blue'

# Create the plot with scaled size and color based on the Daily_Count_Diff_Pct values
filtered_sensors_map = merged_diff_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo=True,
    tiles="CartoLight",
    frame_width=800,
    frame_height=600,
    color=merged_diff_df['Daily_Count_Diff_Pct'].apply(color_map),
    size=merged_diff_df['Daily_Count_Diff_Pct'].apply(scale_size),
    alpha=0.5,
    hover_cols=["Sensor ID", "Sensor Description", "Daily_Count_Diff_Pct"],
    colorbar=True,
)

print(f"Number of sensors in the list: {len(merged_diff_df)}")
filtered_sensors_map


In [ ]:
# Filter rows with "Outdoor" value in "location_type" column
outdoor_pedestrian_sensors_df = sensor_location_df[sensor_location_df["Location Type"] == "Outdoor"]

# List of sensors in Melbourne CBD
#location_ids = [1, 2, 3, 4, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 30, 36, 39, 40, 41, 45, 47, 48, 49, 51, 52, 53, 55, 56, 58, 59, 61, 62, 63, 65, 66, 67, 68, 69, 71, 72, 73, 75, 79, 84, 88, 89, 98, 99, 107, 108, 109, 113] 
location_ids = [1, 3, 4, 17, 19, 23, 49, 58, 59, 65, 66, 79, 109]
cbd_pedestrian_sensors_df = outdoor_pedestrian_sensors_df.loc[outdoor_pedestrian_sensors_df["Sensor ID"].isin(location_ids)]

cbd_pedestrian_sensors_df = cbd_pedestrian_sensors_df.reset_index()
print(f"Number of sensors in the list: {len(cbd_pedestrian_sensors_df)}")
cbd_pedestrian_sensors_df

In [ ]:
cbd_sensors_map = cbd_pedestrian_sensors_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "CartoLight",
    frame_width = 800,
    frame_height = 600,
    color = "Status",
    alpha = 0.5,
    hover_cols=["location_id", "Sensor_description"]
)

print(f"Number of sensors in the list: {len(cbd_pedestrian_sensors_df)}")
cbd_sensors_map